In [3]:
from poloniex import Poloniex
polo = Poloniex()
import os, datetime

#input date time and Symbol
startdate = '1/11/2010'
end_date= '31/12/2018'
Symbol = 'USDT_BTC'



#pull HISTORICAL DATA
#86400 denotes daily
import pandas as pd
Daily_data =polo.returnChartData(Symbol, 86400, start= datetime.datetime.strptime(startdate, "%d/%m/%Y").timestamp(), end= datetime.datetime.strptime(end_date, "%d/%m/%Y").timestamp())
Daily_data = pd.DataFrame(Daily_data)
Daily_data['True Date']= pd.to_datetime(Daily_data['date'], unit='s')
Daily_data['Year']=  Daily_data['True Date'].dt.year
#moving average columns
Daily_data['MA10'] = Daily_data['close'].rolling(10).mean()
Daily_data['MA50'] = Daily_data['close'].rolling(50).mean()
Daily_data['EMA10'] = Daily_data['close'].ewm(span = 10).mean()
Daily_data['EMA50'] = Daily_data['close'].ewm(span = 50).mean()
#Rules
#Signal cross of 10 over 50 Buy at close price Logic 10>50 or short 10<50
#Always gong to be long or short
#Use closing prices
#Profit/Loss -- For buy (10>50) profit = close on day of cross until until 10<50 Reversal.
#Profit/Loss -- For Sell this is the reverse
#charts: Line chart, sma's and closing price
#profit LOSS calculator


Daily_data.head()


,close,date,high,low,open,quoteVolume,volume,weightedAverage,True Date,Year,MA10,MA50,EMA10,EMA50
0,244.00,1424304000,244.000,225.00,225.000000,0.193117,46.276313,239.627778,2015-02-19,2015,NaN,NaN,244.000000,244.000000
1,240.25,1424390400,245.000,240.25,240.250118,0.230429,55.894897,242.568479,2015-02-20,2015,NaN,NaN,241.937500,242.087500
2,245.00,1424476800,245.000,245.00,245.000000,0.060091,14.722239,245.000000,2015-02-21,2015,NaN,NaN,243.168605,243.097420
3,235.00,1424563200,249.000,235.00,245.000000,0.539055,129.121248,239.532608,2015-02-22,2015,NaN,NaN,240.477413,240.950017
4,235.00,1424649600,235.001,235.00,235.000002,0.410926,96.567562,235.000062,2015-02-23,2015,NaN,NaN,238.904997,239.662949


In [4]:
#remove extra data
ema_data = Daily_data[['True Date','close','EMA10','EMA50']].copy()
ema_data.head()

,True Date,close,EMA10,EMA50
0,2015-02-19,244.00,244.000000,244.000000
1,2015-02-20,240.25,241.937500,242.087500
2,2015-02-21,245.00,243.168605,243.097420
3,2015-02-22,235.00,240.477413,240.950017
4,2015-02-23,235.00,238.904997,239.662949


In [51]:
#generate trade signal based on ema10 & ema50 crossing
# also grab p/l for each trade

#signal can equal 0 (no position), 1 (long), -1 (short)
signal = 0
signal_prev = 0
ema_short = 0
ema_long = 0
ema_spread = 0
ema_spread_prev = 0
trade_count = 0

Date = []
Price = []
EMA_short = []
EMA_long = []
spread = []
signals = []
trade_num = []
P_L = []


for index, row in ema_data.iterrows():
    
    ema_short = row['EMA10']
    EMA_short.append(ema_short)
    ema_long = row['EMA50']
    EMA_long.append(ema_long)
    
    ema_spread_prev = ema_spread
    ema_spread = ema_short - ema_long
    spread.append(ema_spread)
    
    date = row['True Date']
    Date.append(date)
    
    price = row['close']
    Price.append(price)
    
    if ema_spread > 0 and ema_spread_prev < 0:
        
        signal_prev = signal
        signal = 1
        signals.append(signal)
        trade_num.append(trade_count)
        trade_count = trade_count +1
        
        
    elif ema_spread < 0 and ema_spread_prev > 0:
        
        signal_prev = signal
        signal = -1
        signals.append(signal)
        trade_num.append(trade_count)
        trade_count = trade_count +1
                   
    
    else:
    
        signals.append(signal)
        
        if signal == 1:
            trade_num.append("LONG")
        elif signal == -1:
            trade_num.append("SHORT")    
        else: 
            trade_num.append("NA")
            
    print(date, price, ema_short, ema_long, ema_spread, signal, trade_count )
    
    




2015-02-19 00:00:00 244.0 244.0 244.0 0.0 0 0
2015-02-20 00:00:00 240.25 241.9375 242.08749999999998 -0.14999999999997726 0 0
2015-02-21 00:00:00 245.0 243.1686046511628 243.09742034395413 0.07118430720868218 1 1
2015-02-22 00:00:00 235.0 240.47741336633666 240.95001749300278 -0.47260412666611273 -1 2
2015-02-23 00:00:00 235.0 238.90499696084393 239.66294913148042 -0.7579521706364858 -1 2
2015-02-24 00:00:00 239.75 239.1244734380544 239.67894633145997 -0.554472893405574 -1 2
2015-02-25 00:00:00 237.75 238.7932805534092 239.3692363476245 -0.5759557942153037 -1 2
2015-02-26 00:00:00 238.5 238.7265578141097 239.2447750618398 -0.5182172477300924 -1 2
2015-02-27 00:00:00 249.0 240.9616993221797 240.5100264900853 0.45167283209440257 1 3
2015-02-28 00:00:00 240.00006025 240.75970110657232 240.44937218895802 0.31032891761429937 1 3
2015-03-01 00:00:00 251.0 242.85166614687924 241.61158712602887 1.2400790208503736 1 3
2015-03-02 00:00:00 269.0 248.0760553939709 244.42873568520525 3.647319708765

2015-07-01 00:00:00 263.05759894 251.90775302730097 242.173331796566 9.734421230734966 1 9
2015-07-02 00:00:00 255.0 252.46997974961107 242.67871244207433 9.791267307536742 1 9
2015-07-03 00:00:00 256.22301049 253.1523489751375 243.2122695039011 9.940079471236402 1 9
2015-07-04 00:00:00 257.0 253.85192188874987 243.75531966548525 10.096602223264625 1 9
2015-07-05 00:00:00 266.99 256.24066336352536 244.6702956496914 11.570367713833946 1 9
2015-07-06 00:00:00 268.03348967 258.38481360106823 245.59018154803903 12.794632053029204 1 9
2015-07-07 00:00:00 262.89154266 259.20421888451096 246.27128577229985 12.932933112211117 1 9
2015-07-08 00:00:00 270.00000001 261.16708818005566 247.20527477110667 13.961813408948984 1 9
2015-07-09 00:00:00 267.1 262.2457994200461 247.98823975414427 14.257559665901852 1 9
2015-07-10 00:00:00 286.0 266.56474498003956 249.48399901403556 17.080745966004002 1 9
2015-07-11 00:00:00 290.00000001 270.8257004400338 251.07808595574525 19.74761448428856 1 9
2015-07-12 

2015-12-28 00:00:00 423.59556697 434.87276683280334 395.08366188435735 39.78910494844598 1 11
2015-12-29 00:00:00 427.00000001 433.4413546832027 396.3352873727139 37.10606731048881 1 11
2015-12-30 00:00:00 425.12 431.9283811044386 397.4641034326016 34.46427767183701 1 11
2015-12-31 00:00:00 430.57315002 431.6819754527225 398.762501615882 32.919473836840496 1 11
2016-01-01 00:00:00 434.98999999 432.28343445950026 400.18319224143204 32.10024221806822 1 11
2016-01-02 00:00:00 436.9499 433.1318827395911 401.6250282230419 31.506854516549197 1 11
2016-01-03 00:00:00 428.14 432.2242676960291 402.6648340205659 29.559433675463197 1 11
2016-01-04 00:00:00 432.00001111 432.18349377129647 403.81523629269435 28.368257478602118 1 11
2016-01-05 00:00:00 430.37677352 431.85499918015165 404.8568679617918 26.998131218359845 1 11
2016-01-06 00:00:00 427.50002018 431.0631848164877 405.74483697456475 25.318347841922957 1 11
2016-01-07 00:00:00 458.89700208 436.1238788643991 407.8292407012047 28.29463816319

2016-04-12 00:00:00 424.89 421.7027137000532 416.0853261370238 5.617387563029411 1 13
2016-04-13 00:00:00 425.0 422.3022203000435 416.43492120811163 5.867299091931841 1 13
2016-04-14 00:00:00 427.17996899 423.1890836982174 416.8562956513563 6.33278804686114 1 13
2016-04-15 00:00:00 430.61709605 424.5396313985415 417.39593490780805 7.143696490733419 1 13
2016-04-16 00:00:00 430.85 425.6869711442612 417.92354532718895 7.763425817072232 1 13
2016-04-17 00:00:00 428.0 426.1075218453046 418.31870042938556 7.788821415919017 1 13
2016-04-18 00:00:00 430.31052983 426.8717051152492 418.7889682684422 8.082736846806995 1 13
2016-04-19 00:00:00 437.93750001 428.88366782338574 419.5398911116376 9.343776711748149 1 13
2016-04-20 00:00:00 443.12202632 431.4724602773156 420.4646807629136 11.007779514401989 1 13
2016-04-21 00:00:00 450.79900002 434.9863765941673 421.65426195382975 13.33211464033758 1 13
2016-04-22 00:00:00 447.539519 437.2687661225005 422.66937010901677 14.599396013483727 1 13
2016-04-

2016-09-02 00:00:00 576.01000003 576.5605104494501 595.5124961393315 -18.951985689881326 -1 14
2016-09-03 00:00:00 610.41999481 582.7167803331864 596.0971039304347 -13.38032359724832 -1 14
2016-09-04 00:00:00 610.65 587.7955475453343 596.6678057371751 -8.872258191840729 -1 14
2016-09-05 00:00:00 610.10000541 591.850903520728 597.1945586656016 -5.343655144873537 -1 14
2016-09-06 00:00:00 614.74713894 596.0138554151412 597.8828951470515 -1.869039731910334 -1 14
2016-09-07 00:00:00 619.72886368 600.325675099661 598.7395997955231 1.5860753041379212 1 15
2016-09-08 00:00:00 634.26777916 606.4969667469953 600.1328617315736 6.364105015421728 1 15
2016-09-09 00:00:00 627.00000002 610.2247909784508 601.1864749979235 9.038315980527273 1 15
2016-09-10 00:00:00 627.99998771 613.4566449296415 602.2379853004892 11.218659629152285 1 15
2016-09-11 00:00:00 610.51369403 612.9215629478884 602.5625228977639 10.359040050124577 1 15
2016-09-12 00:00:00 611.99999999 612.7540060464542 602.9326200386785 9.821

2017-01-22 00:00:00 924.11939955 892.8596948780543 862.1926403807242 30.667054497330014 1 15
2017-01-23 00:00:00 912.22165 896.3800503547717 864.1545623265793 32.225488028192444 1 15
2017-01-24 00:00:00 885.00000008 894.3109503048132 864.9720304737726 29.338919831040585 1 15
2017-01-25 00:00:00 892.66777263 894.0121907275744 866.0581380093115 27.954052718262915 1 15
2017-01-26 00:00:00 914.09767069 897.6640961752881 867.9420412516924 29.722054923595692 1 15
2017-01-27 00:00:00 917.02745093 901.1847061306903 869.8669592782937 31.317746852396567 1 15
2017-01-28 00:00:00 918.58600386 904.3485784451103 871.7775100462047 32.571068398905595 1 15
2017-01-29 00:00:00 914.9998 906.285164182363 873.4725018090994 32.812662373263606 1 15
2017-01-30 00:00:00 918.0 908.4151343310242 875.2186782087433 33.19645612228089 1 15
2017-01-31 00:00:00 967.12176383 919.0890669672017 878.8227207821275 40.26634618507421 1 15
2017-02-01 00:00:00 983.0 930.7092366095287 882.9081042808692 47.80113232865949 1 15
20

2017-08-25 00:00:00 4352.00000091 4131.7487722427295 3341.2956338550835 790.453138387646 1 19
2017-08-26 00:00:00 4354.95 4172.330813653143 3381.0467854686094 791.2840281845338 1 19
2017-08-27 00:00:00 4321.66862636 4199.483143236208 3417.9339164839585 781.5492267522495 1 19
2017-08-28 00:00:00 4380.009999 4232.306207920533 3455.662390308117 776.6438176124157 1 19
2017-08-29 00:00:00 4590.00000001 4297.341442845891 3500.1462181395636 797.1952247063273 1 19
2017-08-30 00:00:00 4573.3974586 4347.5334457102745 3542.2345020791886 805.2989436310859 1 19
2017-08-31 00:00:00 4715.15472512 4414.373678330225 3588.231373570985 826.1423047592398 1 19
2017-09-01 00:00:00 4899.00000047 4502.487555082911 3639.6340648219266 862.8534902609845 1 19
2017-09-02 00:00:00 4511.0 4504.035272340564 3673.8052779661652 830.2299943743988 1 19
2017-09-03 00:00:00 4604.23005 4522.252504642279 3710.2925239282763 811.9599807140025 1 19
2017-09-04 00:00:00 4208.99999996 4465.2975037909555 3729.8496798510887 735.4478

2018-01-14 00:00:00 13560.0 14375.427279031861 13577.942594158749 797.4846848731122 1 21
2018-01-15 00:00:00 13580.0 14230.804137389703 13578.023276740758 652.7808606489452 1 21
2018-01-16 00:00:00 11102.69798098 13662.05756349703 13480.951696514845 181.10586698218503 1 21
2018-01-17 00:00:00 11036.11532467 13184.613520073932 13385.075760364067 -200.46224029013501 -1 22
2018-01-18 00:00:00 11010.00009 12789.229260060489 13291.935537996847 -502.7062779363587 -1 22
2018-01-19 00:00:00 11410.53924928 12538.55834900949 13218.1552913805 -679.5969423710103 -1 22
2018-01-20 00:00:00 12750.17487957 12577.034081838674 13199.803118368325 -622.7690365296512 -1 22
2018-01-21 00:00:00 11510.0 12383.027885140733 13133.536329412704 -750.5084442719708 -1 22
2018-01-22 00:00:00 10767.0 12089.204633296962 13040.730983161226 -951.5263498642635 -1 22
2018-01-23 00:00:00 10807.7699848 11856.216515388423 12953.163885186277 -1096.9473697978538 -1 22
2018-01-24 00:00:00 11388.8 11771.23169440871 12891.8162818

2018-06-06 00:00:00 7650.00000217 7596.099453831273 8138.671703855869 -542.5722500245956 -1 26
2018-06-07 00:00:00 7694.09351679 7613.916556187406 8121.237265147403 -507.3207089599973 -1 26
2018-06-08 00:00:00 7610.62165072 7613.317482466059 8101.213123405151 -487.8956409390921 -1 26
2018-06-09 00:00:00 7490.0 7590.896122017684 8077.243981310831 -486.34785929314694 -1 26
2018-06-10 00:00:00 6763.91319959 7440.535590667197 8025.740813400211 -585.2052227330141 -1 26
2018-06-11 00:00:00 6874.596078 7337.637497454979 7980.597882600203 -642.9603851452239 -1 26
2018-06-12 00:00:00 6540.00000001 7192.612497919528 7924.103847988823 -731.4913500692946 -1 26
2018-06-13 00:00:00 6292.30084274 7028.919469705069 7860.111573273182 -831.1921035681135 -1 26
2018-06-14 00:00:00 6626.9286401 6955.830227958692 7811.751458246783 -855.9212302880906 -1 26
2018-06-15 00:00:00 6376.15122659 6850.434045891658 7755.453409946517 -905.0193640548596 -1 26
2018-06-16 00:00:00 6483.83962365 6783.780514574993 7705.58

2018-10-27 00:00:00 6502.28201824 6552.206057062852 6591.941169289893 -39.7351122270411 -1 30
2018-10-28 00:00:00 6495.0 6541.804955778696 6588.139554807936 -46.33459902924005 -1 30
2018-10-29 00:00:00 6350.0 6506.931327455298 6578.800748737037 -71.86942128173905 -1 30
2018-10-30 00:00:00 6332.23599571 6475.168539865244 6569.131542735977 -93.96300287073245 -1 30
2018-10-31 00:00:00 6376.30189951 6457.1927870733825 6561.569595942802 -104.37680886941962 -1 30
2018-11-01 00:00:00 6405.26712993 6447.751758501859 6555.440087471711 -107.68832896985259 -1 30
2018-11-02 00:00:00 6435.40757191 6445.507360939702 6550.732929998703 -105.22556905900092 -1 30
2018-11-03 00:00:00 6387.09989981 6434.887822552484 6544.315948422676 -109.42812587019216 -1 30
2018-11-04 00:00:00 6482.0 6443.453672997487 6541.872185739434 -98.41851274194687 -1 30
2018-11-05 00:00:00 6471.38864399 6448.53275863249 6539.108125278673 -90.57536664618237 -1 30
2018-11-06 00:00:00 6518.77235989 6461.303595224765 6538.31064428303

In [53]:
ema_dict = {'Date':Date,
           'Price':Price,
           'EMA Short':EMA_short,
           'EMA Long': EMA_long,
           'EMA Spread': spread,
           'Signal':signals,
           'Trade':trade_num}

ema_strategy = pd.DataFrame(ema_dict)

ema_strategy.head(100)

,Date,Price,EMA Short,EMA Long,EMA Spread,Signal,Trade
0,2015-02-19,244.000000,244.000000,244.000000,0.000000,0,NA
1,2015-02-20,240.250000,241.937500,242.087500,-0.150000,0,NA
2,2015-02-21,245.000000,243.168605,243.097420,0.071184,1,0
3,2015-02-22,235.000000,240.477413,240.950017,-0.472604,-1,1
4,2015-02-23,235.000000,238.904997,239.662949,-0.757952,-1,SHORT
5,2015-02-24,239.750000,239.124473,239.678946,-0.554473,-1,SHORT
6,2015-02-25,237.750000,238.793281,239.369236,-0.575956,-1,SHORT
7,2015-02-26,238.500000,238.726558,239.244775,-0.518217,-1,SHORT
8,2015-02-27,249.000000,240.961699,240.510026,0.451673,1,2
9,2015-02-28,240.000060,240.759701,240.449372,0.310329,1,LONG


In [57]:
trade_count

30

In [34]:
#id = (ema_strategy.Trade.values = 3).argmax()
#ema_strategy.sort_values(by=['Trade'])
#ema_strategy.Trade.searchsorted('5', side='right')

index_prev
prev_close


for index in range(len(ema_strategy['Trade'])):
    
    if ema_strategy['Trade'][index] != ema_strategy['Trade'][index_prev] & 
    ema_strategy['Trade'][signal] != ema_strategy['Trade']signal_prev:
        
        break

8


In [66]:
print(ema_strategy.loc[ema_strategy['Trade'].isin([1])])

        Date  Price   EMA Short    EMA Long  EMA Spread  Signal Trade
3 2015-02-22  235.0  240.477413  240.950017   -0.472604      -1     1


In [68]:
ema_strategy.loc[ema_strategy['Trade'] < trade_count]

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
import